In [1]:
import cv2
import numpy as np
import os

In [2]:
# Data

dataset_path = "./data/"
faceData = []
labels = []
nameMap = {}

classId = 0

for f in os.listdir(dataset_path):
    if f.endswith(".npy"):
        nameMap[classId] = f[:-4]
        # X Value
        dataItem = np.load(dataset_path + f)
        m = dataItem.shape[0]
        faceData.append(dataItem)
        
        # Y Value
        target = classId * np.ones((m,))
        classId += 1
        labels.append(target)
        
        

XT = np.concatenate(faceData,axis=0)
YT = np.concatenate(labels,axis=0)

In [3]:
XT.shape, YT.shape

In [4]:
921600/(3*300)

In [5]:

# KNN Algorithm 

def dist(p, q):
    return np.sqrt(np.sum((p - q) ** 2))


def knn(XT,YT,xt,k=5):
    
    m = XT.shape[0]
    dlist = []
    
    for i in range(m):
        d = dist(XT[i],xt)
        dlist.append((d,YT[i]))
        
    dlist = sorted(dlist)
    dlist = np.array(dlist[:k])
    labels = dlist[:,1]
    
    
    labels, cnts = np.unique(labels,return_counts=True)
    idx = cnts.argmax()
    pred = labels[idx]
    
    return int(pred)

   

In [6]:

# Predictions

cam = cv2.VideoCapture(0)

model = cv2.CascadeClassifier("raw.githubusercontent.com_opencv_opencv_4.x_data_haarcascades_haarcascade_frontalface_alt.xml")
offset =10
while True:
    success , img = cam.read()
    if not success:
        print("The Process failed")    
    
    faces = model.detectMultiScale(img,1.3,5)
    
    for f in faces:
        x,y,w,h = f
        cropped_face = img[y - offset:y+h + offset , x - offset:x + offset + w]
        cropped_face = cv2.resize(cropped_face,(100,100))
        
        # Predict using KNN
        classPredicted = knn(XT,YT,cropped_face.flatten())
        
        namePredicted = nameMap[classPredicted]
        
        # Display Name and box
        cv2.putText(img,namePredicted,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    
    cv2.imshow("Prediction Window",img)
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
        
        
cam.release()
cv2.destroyAllWindows()